In [5]:
%pip install torchvision datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from datasets import load_dataset
from torchvision import datasets

In [7]:
datasets.CIFAR10(root='~/data/10', download=True)

Files already downloaded and verified


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: /home/jovyan/data/10
    Split: Train

In [8]:
datasets.CIFAR100(root='~/data/100', download=True)

Files already downloaded and verified


Dataset CIFAR100
    Number of datapoints: 50000
    Root location: /home/jovyan/data/100
    Split: Train

In [9]:
sst2_train = load_dataset('stanfordnlp/sst2', split='train')
sst2_eval = load_dataset('stanfordnlp/sst2', split='validation')
sst2_test_blank = load_dataset('stanfordnlp/sst2', split='test')

In [10]:
data = sst2_train.train_test_split(test_size=0.2, seed=42, stratify_by_column='label')

In [11]:
sst2_train = data['train']
sst2_test = data['test']

In [12]:
sst2_train.save_to_disk("~/data/sst2/train")
sst2_eval.save_to_disk("~/data/sst2/eval")
sst2_test.save_to_disk("~/data/sst2/test")

sst2_test_blank.save_to_disk("~/data/sst2/test-blank")

Saving the dataset (0/1 shards):   0%|          | 0/53879 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/872 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13470 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1821 [00:00<?, ? examples/s]

In [13]:
dbpedia_train = load_dataset("fancyzhx/dbpedia_14", split='train')
dbpedia_test = load_dataset("fancyzhx/dbpedia_14", split='test') 
data = dbpedia_train.train_test_split(test_size=0.2, seed=42, stratify_by_column='label')

In [14]:
dbpedia_train = data['train']
dbpedia_eval = data['test']

dbpedia_train = dbpedia_train.rename_column("content", "sentence")
dbpedia_train = dbpedia_train.remove_columns(["title"])

dbpedia_eval = dbpedia_eval.rename_column("content", "sentence")
dbpedia_eval = dbpedia_eval.remove_columns(["title"])

dbpedia_test = dbpedia_test.rename_column("content", "sentence")
dbpedia_test = dbpedia_test.remove_columns(["title"])

In [15]:
dbpedia_train.save_to_disk("~/data/dbpedia/train")
dbpedia_eval.save_to_disk("~/data/dbpedia/eval")
dbpedia_test.save_to_disk("~/data/dbpedia/test")

Saving the dataset (0/1 shards):   0%|          | 0/448000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/112000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/70000 [00:00<?, ? examples/s]

In [16]:
trec_train_coarse = load_dataset("CogComp/trec", split='train') 
trec_test_coarse = load_dataset("CogComp/trec", split='test') 

data = trec_train_coarse.train_test_split(test_size=0.2, seed=42, stratify_by_column='coarse_label')

In [17]:
trec_train_coarse = data['train']
trec_eval_coarse = data['test']

trec_train_coarse = trec_train_coarse.rename_column("text", "sentence")
trec_train_coarse = trec_train_coarse.rename_column("coarse_label", "label")
trec_train_coarse = trec_train_coarse.remove_columns(["fine_label"])

trec_eval_coarse = trec_eval_coarse.rename_column("text", "sentence")
trec_eval_coarse = trec_eval_coarse.rename_column("coarse_label", "label")
trec_eval_coarse = trec_eval_coarse.remove_columns(["fine_label"])

trec_test_coarse = trec_test_coarse.rename_column("text", "sentence")
trec_test_coarse = trec_test_coarse.rename_column("coarse_label", "label")
trec_test_coarse = trec_test_coarse.remove_columns(["fine_label"])

In [19]:
trec_train_coarse.save_to_disk("~/data/trec/train_coarse")
trec_eval_coarse.save_to_disk("~/data/trec/eval_coarse")
trec_test_coarse.save_to_disk("~/data/trec/test_coarse")

Saving the dataset (0/1 shards):   0%|          | 0/4361 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1091 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

In [20]:
trec_train_coarse = load_dataset("CogComp/trec", split='train') 
trec_test_coarse = load_dataset("CogComp/trec", split='test') 

data = trec_train_coarse.train_test_split(test_size=0.2, seed=42, stratify_by_column='fine_label')

In [21]:
trec_train_coarse = data['train']
trec_eval_coarse = data['test']

trec_train_coarse = trec_train_coarse.rename_column("text", "sentence")
trec_train_coarse = trec_train_coarse.rename_column("fine_label", "label")
trec_train_coarse = trec_train_coarse.remove_columns(["coarse_label"])

trec_eval_coarse = trec_eval_coarse.rename_column("text", "sentence")
trec_eval_coarse = trec_eval_coarse.rename_column("fine_label", "label")
trec_eval_coarse = trec_eval_coarse.remove_columns(["coarse_label"])

trec_test_coarse = trec_test_coarse.rename_column("text", "sentence")
trec_test_coarse = trec_test_coarse.rename_column("fine_label", "label")
trec_test_coarse = trec_test_coarse.remove_columns(["coarse_label"])

In [22]:
trec_train_coarse.save_to_disk("~/data/trec/train_fine")
trec_eval_coarse.save_to_disk("~/data/trec/eval_fine")
trec_test_coarse.save_to_disk("~/data/trec/test_fine")

Saving the dataset (0/1 shards):   0%|          | 0/4361 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1091 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]